In [1]:
%load_ext autoreload
%autoreload 2

from solution import matrix_to_sycamore_operations

import cirq
from cirq.circuits import Circuit
import numpy as np
from scipy.stats import unitary_group
from cirq.sim import Simulator
sim = Simulator()

In [2]:
# Example
num_qubits = 4
qubits =[cirq.GridQubit(0,i) for i in range(num_qubits)]

A = unitary_group.rvs(2**num_qubits)

In [6]:
# Example
ops, _ = matrix_to_sycamore_operations(qubits, A)

circuit = Circuit(ops)
print(circuit.to_text_diagram(transpose=True))

(0, 0)                          (0, 1)                            (0, 2)                           (0, 3)
│                               │                                 │                                │
@───────────────────────────────@─────────────────────────────────@────────────────────────────────PhXZ(a=-0.795,x=0.44,z=0.526)
│                               │                                 │                                │
@───────────────────────────────@─────────────────────────────────(0)──────────────────────────────X
│                               │                                 │                                │
@───────────────────────────────@─────────────────────────────────PhXZ(a=-0.393,x=0.67,z=-0.214)───(0)
│                               │                                 │                                │
@───────────────────────────────@─────────────────────────────────(0)──────────────────────────────X
│                               │                       

In [31]:
cirq.unitary(ops[0].gate.sub_gate)

array([[ 0.59444923+0.49099831j, -0.03045692+0.63609995j],
       [-0.09957222-0.62899614j, -0.53742205+0.55283521j]])

In [27]:
cirq.unitary(cirq.PhasedXZGate(x_exponent=0, z_exponent=1, axis_phase_exponent=1))

array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]])